In [34]:
from Crypto.Util.number import sieve_base, inverse, GCD
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
## Helper functions

def generate_private_key(p):
    secret_key = random.randint(1, p-1)
    return secret_key
    
def generate_public_key(g, secret_key, p):
    return pow(g, secret_key, p)

# Prerequisites

- DLP and its vulnerabilities
- DH basics
- Group theory

# Theory + Attacks

- https://en.wikipedia.org/wiki/Man-in-the-middle_attack
- https://stackoverflow.com/questions/10471009/how-does-the-man-in-the-middle-attack-work-in-diffie-hellman
- Computerphile https://www.youtube.com/watch?v=vsXMMT2CqqE&t=276s&ab_channel=Computerphile

Basic Diffie Hellman key exchange can be broken if Eve is in the middle of the conversation and mingles with parameters 
We will present some ways this can happen

# Attacks

In [12]:
g = 2
#Good small-sized p
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
p.bit_length()

1536

## If you can intercepted Alice and can send parameters to Bob

You know $A, p, g$  
Send to Bob:
- p = p
- g = A
- A = A

Bob will compute 
- $B = A^b = g^{ab}$ as his public key that you will know
- $A^b = g^{ab}$ as the secret key

Therefore you'll know $B$ as the secret_key and send it back to Alice

**Note** This method works to decrypt past messages too if Bob uses a *static* key (doesn't change it)

In [32]:
a = generate_private_key(p) # you don't know this
A = generate_public_key(g, a, p) # you know this

#Send p = p, g = A, A = A
b = generate_private_key(p) # you don't know this
B = generate_public_key(A, b, p) # you know this

#secret
shared_secret = pow(A, b, p)

In [33]:
shared_secret == B

True

## You can only change the prime $p$

- Suppose Alice and Bob exchanged parameters once already $(p, g)$. You can send $(p', g')$

If Bob verifies $g = g'$ and you are allowed only to change the prime $p$ then you can setup a group weak to Pohlig-Hellman DLP
- if $p$ is strong => $p = 2q+1$ with $q$ prime => $p-1$ is divisible only by $2$ and some big prime
- make your $p' = 2 \cdot \Pi_{k=1}^{n} P_i^{e_i} + 1$ for some $n$ you chose and some small primes $P_i$ of your choice. This group is vulnerable to Pohlig-Hellman since $p'-1$ is smooth. The order of 2 must be the same order as in $p-1$ to ensure the solutions are the same
- If Bob uses this group to compute $g^b = B \ mod \ p' $ Then you can efficiently solve the DLP and find his $b$

**Note** This method works to decrypt past messages too if Bob uses a *static* key (doesn't change it)

# Downgrading

*Idea*: 
- Before exchanging messages Bob and Alice need to exchange the security protocol (bitsize / type of encryption)
- Eve can mingle with these negotiations and downgrade the encryption

Logjam attack: 
- https://www.youtube.com/watch?v=Qi8ynAB2dpM&ab_channel=DevanSingh
- https://en.wikipedia.org/wiki/Logjam_(computer_security)
- https://cryptologie.net/article/270/the-logjam-attack/
        